## Dot (异常状态) 系统重构规划

Dot (Damage over Time) 系统目前由 `BaseDot` 类及其子类（如 `Shock`, `Ignite`）管理。在旧架构中，它独立于 Buff 系统，拥有自己的一套 `start`, `end`, `ready_judge` 生命周期。
在新架构中，**异常状态（Anomaly）本质上就是一种 Buff**，其“周期性造成伤害”的特性可以通过新系统的 `TriggerEffect`（触发器效果）完美实现。

因此，本次重构将 **废弃独立的 Dot 模块**，将其完全通过 `Buff` + `EventRouter` 的形式重写。

### 1. 核心映射关系 (Mapping)

| 旧 Dot 概念 | 新系统对应实现 | 说明 |
| :--- | :--- | :--- |
| **`BaseDot` 类** | **`Buff` 模型** | Dot 不再有独立的类，而是作为 ID 特定的通用 `Buff` 实例存在（如 `Buff(id=9001, name="感电")`）。 |
| **`ft.update_cd`** (触发间隔) | **`PeriodicTrigger`** | 在 `TriggerEffect` 中增加基于时间的触发条件，或在 `Schedule` 系统中注册周期性事件。 |
| **`ft.max_duration`** | **`Buff.duration`** | 通用的 Buff 持续时间属性。 |
| **`ft.effect_rules`** (效果逻辑) | **`Effect` 组合** | 使用 `TriggerEffect` 组合 `Condition` (时间到了) 和 `Action` (造成异常伤害)。 |
| **`dy`** (动态数据) | **`Buff.dy`** | 通用的 Buff 动态数据容器。 |
| **`AnomalyBar` 交互** | **`EventHandler`** | 异常积蓄槽满时，不再 `new Shock()`，而是 `buff_manager.add_buff("Shock")`。 |

### 2. 详细施工步骤

#### 阶段 A: 基础设施扩展 (Infrastructure)

1.  **扩展 `TriggerEffect` 支持周期性事件**:
    * 在 `zsim/sim_progress/Buff/effects/trigger_effect.py` 中，设计一种机制来处理“每隔 X 秒触发一次”的逻辑。
    * 事件驱动: `BuffManager` 在添加 Buff 时，若识别到其含有周期性 Tag，则向全局时间调度器 (`Schedule`) 注册一个循环事件 `PeriodicBuffTick(buff_id)`。

2.  **实现 `AnomalyDamageAction`**:
    * 在 `zsim/sim_progress/Buff/effects/actions.py` (需新建) 中实现 `DealAnomalyDamage` 行为。
    * 该行为需要能够读取快照属性（因为部分异常伤害可能基于触发时的面板快照，也可能基于实时面板，需根据游戏机制确定）。

#### 阶段 B: 数据迁移 (Migration)

1.  **配置化转化**:
    * 分析 `Shock.py`, `Ignite.py` 等文件中的 `DotFeature`。
    * 将其转化为 `GlobalBuffController` 的配置项（JSON 或工厂配置）。
    * *示例配置*:
        ```python
        # Shock (感电) 配置示例
        BuffConfig(
            id=9001,
            name="感电",
            duration=10.0,
            max_layers=1,
            effects=[
                # 效果1: 周期性伤害
                TriggerEffect(
                    trigger=PeriodicTimer(interval=1.0),
                    action=DealAnomalyDamage(type=DmgType.ELECTRIC, multiplier=1.25)
                ),
                # 效果2: 间断 (紊乱前置) - 这是一个被动特性，可能通过 Tag 实现
                TagEffect(tags=["Anomaly", "Electric", "Shock"])
            ]
        )
        ```

#### 阶段 C: 逻辑接入 (Integration)

1.  **改造 `AnomalyBar` (`zsim/sim_progress/anomaly_bar/`)**:
    * 修改 `Accumulate()` 或 `Trigger()` 逻辑。
    * **删除** 对 `BaseDot` 的引用。
    * **新增** 调用 `target_char.buff_manager.add_buff(anomaly_buff_id)`。

2.  **处理紊乱 (Disorder)**:
    * 紊乱本质上是：当添加一个新的异常 Buff 时，检测是否存在不同属性的异常 Buff。
    * 这完全可以通过 `EventRouter` 实现：
        * **监听**: `BUFF_ADDED` 事件。
        * **条件**: `AddedBuff.has_tag("Anomaly")` AND `Target.has_buff_with_tag("Anomaly")` AND `Types_Are_Different`.
        * **行为**: `TriggerDisorder()` (结算剩余伤害，清除原 Buff)。